## Assignment 1: Bayes Classifier Question 3

In this question, you will implement the **CategoricaL Naive Bayes classifier** from scratch.
This means you must not rely on any pre-implemented models (such as those provided
by the scikit-learn library), in order to gain a deeper understanding of how the
algorithm works internally. 

Through this process, you will explore how probabilities are estimated from data, 
how predictions are made using conditional independence assumptions, and how numerical 
issues can arise in probabilistic models.

Unless explicitly stated otherwise, you should not be making any changes to the library
imports section of this notebook. In particular, you are not allowed to import any additional Python libraries. 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

**Data Loader and Processing Section**

In [ ]:
# =====================================================================================
# TODO: Using Numpy, extract CSV data. Note: Please omit header row.

# *Add code here*

# =====================================================================================
# TODO: Split dataset into a feature matrix and label vector.

# *Add code here*

# =====================================================================================
# TODO: Convert categorical strings to intergers for each feature. 
# (Example Low, Medium, High -> 0, 1, 2)

# *Add code here*

# =====================================================================================
# TODO: Use train_test_split function from the Scikit-learn library to split data 
# into training (75%) and test (25%) sets.
# *IMPORTANT: Add the arguemnt random_state=42*
# Hint: Use stratify=y to maintain class distribution

# *Add code here*

**Categorical Naive Bayes Classifier**

Write your code where you see the key word pass. (You should remove pass)

In [ ]:
class CategoricalNaiveBayes:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.classes = None             # Unique class labels
        self.priors = None              # Class prior probabilities
        self.feature_cond_probs = None  # List to store feature probabilities per class

    def fit(self, X, y):
        """
        TODO:
        - Identify unique classes
        - Compute class priors P(y)
        - Count occurrences of each feature value per class
        - Apply Laplace smoothing (alpha)
        """
        pass

    def compute_log_likelihood(self, X):
        """
        TODO:
        - For each sample and each class:
        - Sum log-probabilities of all features given the class
        - Add log prior of the class
        - Return a log-likelihood array of shape (n_samples, n_classes)
        """
        pass

    def predict(self, X):
        """
        TODO:
        - Use compute_log_likelihood
        - Return the class with highest log-posterior for each sample
        """
        pass

**Classifier training and testing**

In [ ]:
# =====================================================================================
# TODO: Create an instance of your CategoricalNaiveBayes classifier and train it

# *Add code here*

# =====================================================================================
# TODO: Make predictions on the test data

# *Add code here*

# =====================================================================================
# TODO: Using Scikit-learn's accuracy_score and classification_report functions, print
# both the accuracry score and the classification_report metrics for your classifier

# *Add code here*